In [3]:
import pandas as pd 
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import os

In [1]:
rs = 1

In [13]:
def make_params(string0,string1,string2,string3=None):
    params = []
    if string3:
        for el2 in string3[1]:
            for el in string2[1]:
                params.append({string1[0]:string1[1],
                              string0[0]+'__'+string2[0]:[el],
                              string0[1]+'__estimator__'+string2[0]:[el],
                              string0[0]+'__'+string3[0]:[el2],
                              string0[1]+'__estimator__'+string3[0]:[el2]})
    else:
        for el in string2[1]:
            params.append({string1[0]:string1[1],
                         string0[0]+'__'+string2[0]:[el],
                         string0[1]+'__estimator__'+string2[0]:[el]})
    return params

In [6]:
def read_data(file):
    """
    
    """
    df = (pd.read_csv(file)
            .rename(columns=str.lower)
            .set_index("id")
        )
    return df
    

In [7]:
data = read_data("../data/train.csv")
X_train = data.drop(columns='label')
y_train = data.label
data.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f16,f17,f18,f19,f20,f21,f22,f23,f24,label
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,1.0,179.0,99.0,99.0,139.0,74.5822,301.0590,53.6656,...,0.0,0.0,99.0,0.1788,0.3583,0.0132,7.0,2.0,2.0,0.0
1,1.0,0.0,1.0,298.0,258.0,258.0,239.0,181.8275,135.3403,179.4714,...,5290.0,0.0,0.0,0.4355,0.3375,0.0458,5.0,2.0,4.0,2.0
2,1.0,1.0,1.0,99.0,139.0,139.0,159.0,83.0692,187.6926,149.9667,...,0.0,1.0,2442.0,0.0762,0.1167,0.0200,8.0,6.0,1.0,2.0
3,1.0,1.0,0.0,238.0,139.0,139.0,258.0,248.3948,505.5022,231.0000,...,397.0,0.0,0.0,0.2176,0.3791,0.0139,5.0,8.0,5.0,1.0
4,1.0,1.0,0.0,358.0,179.0,179.0,219.0,127.8251,117.0438,44.0114,...,0.0,0.0,0.0,0.2923,0.2908,0.0539,3.0,1.0,3.0,0.0


## presettings

In [4]:
cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=rs)
cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=rs)
cv_sfs = StratifiedKFold(n_splits=5, shuffle=True, random_state=rs)

In [8]:
x_temp = PolynomialFeatures().fit_transform(X_train)

## logistic regression

In [16]:
## LR
est = LogisticRegression()
sfs1 = SFS(estimator=est, 
           k_features=3,
           forward=False, 
           floating=False, 
           scoring='accuracy',
           cv=cv_sfs)

pipe_lr_sfs = Pipeline([('inter', PolynomialFeatures()),
                         ('sc',  StandardScaler()),
                         ('sfs', sfs1),
                         ('lr',est)])

string1 = ['sfs__k_features', [(10,325)]]
string2 = ['C',[0.001,0.01,0.1,10,100,1000]]
string3 = ['penalty',['l1','l2']]
param_grid_lr_sfs = make_params(['lr','sfs'],string1,string2,string3)

gs_lr_sfs = GridSearchCV(estimator = pipe_lr_sfs,
                  param_grid = param_grid_lr_sfs,
                  cv = cv_inner,
                  scoring = "accuracy",
                  n_jobs=-1,
                  verbose=2)

In [ ]:
nested_cv_lr = cross_val_score(gs_lr_sfs,X_train,y_train,cv=cv_outer)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
## RFR
est = RandomForestClassifier(n_estimators=10,random_state=rs)
sfs1 = SFS(estimator=est, 
           k_features=3,
           forward=True, 
           floating=False, 
           scoring='neg_mean_absolute_error',
           cv=cv_sfs)

pipe_rfr_sfs = Pipeline([('pre_pros',  preprocessor),
                 ('sfs', sfs1),
                 ('rfc',est)])

string1 = ['sfs__k_features', [(2,no_feat)]]
string2 = ['n_estimators',[1,5,10,50,100,250,500,1000]]
string3 = ['max_depth',[2,5,10,20,50,75,150]]
param_grid_rfr_sfs = make_params(['rfc','sfs'],string1,string2,string3)

gs_rf_sfs = GridSearchCV(estimator = pipe_rfr_sfs,
                  param_grid = param_grid_rfr_sfs,
                  cv = cv_inner,
                  scoring = "accuracy",
                  n_jobs=-1,
                  verbose=2)